In [1]:
import pyspark

In [34]:
# (date, user_id, song_id, song_title, artist)
canciones = [
    ('2020-05-23',1,1,'Womanizer','Britney Spears'),
    ('2020-05-23',2,10,'Physical','Dua Lipa'),
    ('2020-05-23',3,1,'Womanizer','Britney Spears'),
    ('2020-05-23',4,11,'Wrecking Ball','Miley Cyrus'),
    ('2020-05-23',5,10,'Physical','Dua Lipa'),
    ('2020-05-24',1,2,'Just luv me', 'Britney Spears'),
    ('2020-05-24',2,1,'Womanizer', 'Britney Spears'),
    ('2020-05-24',3,1,'Womanizer', 'Britney Spears'),
    ('2020-05-24',4,1,'Womanizer', 'Britney Spears'),
    ('2020-05-24',5,2,'Just luv me', 'Britney Spears'),
    ('2020-05-25',1,3,'Circus','Britney Spears'),
    ('2020-05-25',2,2,'Just luv me','Britney Spears'),
    ('2020-05-25',3,2,'Just luv me','Britney Spears'),
    ('2020-05-25',4,2,'Just luv me','Britney Spears'),
    ('2020-05-25',5,3,'Circus','Britney Spears'),
    ('2020-05-26',1,4,'Work B*tch', 'Britney Spears'),
    ('2020-05-26',2,3,'Circus', 'Britney Spears'),
    ('2020-05-26',3,4,'Work B*tch', 'Britney Spears'),
    ('2020-05-26',4,3,'Circus', 'Britney Spears'),
    ('2020-05-26',5,4,'Work B*tch', 'Britney Spears')
]

# (song_id, tag)
tags = [
    (1,'pop'),
    (2,'acoustic'),
    (10,'actual'),
    (10,'pop'),
    (1,'britney spears'),
    (10,'top10'),
    (2,'britney spears'),
    (11,'2010s'),
    (10,'rising'),
    (3,'pop'),
    (3,'2010s'),
    (4,'pop'),
    (4,'britney spears'),
    (4,'2010s')
]

Spotify cuenta con un log de todas las canciones que fueron escuchadas en su  plataforma,  esta  información  se  encuentra  en  un RDD  que  está paralelizado por día, es decir cada día es una partición. Los campos del RDD son los siguientes: (date, user_id, song_id, song_title, artist).
Se cuenta por otro lado con un RDD que asigna "tags" a las canciones, por ejemplo "rock, punk, actual, top-10, acoustic, etc). Una canción puede tener asociados "n" tags. El RDD tiene el formato (song_id, tag). 
Cada día se corre un proceso para asignar el tag "rising" a las canciones que se escucharon mas veces el día de hoy que el día de ayer. Estos nuevos tags pasan  a  formar  parte  del  RDD  de  tags  para  futuros  usos.  Programar  en PySpark usando el API de RDD lo solicitado.

In [35]:
cancionesRdd = sc.parallelize(canciones)
tagsRdd = sc.parallelize(tags)

In [43]:
# mapeo rdd de canciones tal que quede ((song_id, fecha),contador)
cancionesRddMapped = cancionesRdd.map(lambda x:((x[2],x[0]),1))
cancionesRddMapped.collect()

[((1, '2020-05-23'), 1),
 ((10, '2020-05-23'), 1),
 ((1, '2020-05-23'), 1),
 ((11, '2020-05-23'), 1),
 ((10, '2020-05-23'), 1),
 ((2, '2020-05-24'), 1),
 ((1, '2020-05-24'), 1),
 ((1, '2020-05-24'), 1),
 ((1, '2020-05-24'), 1),
 ((2, '2020-05-24'), 1),
 ((3, '2020-05-25'), 1),
 ((2, '2020-05-25'), 1),
 ((2, '2020-05-25'), 1),
 ((2, '2020-05-25'), 1),
 ((3, '2020-05-25'), 1),
 ((4, '2020-05-26'), 1),
 ((3, '2020-05-26'), 1),
 ((4, '2020-05-26'), 1),
 ((3, '2020-05-26'), 1),
 ((4, '2020-05-26'), 1)]

In [44]:
cancionesRddReduced = cancionesRddMapped.reduceByKey(lambda x,y: x+y)
cancionesRddReduced.collect()

[((3, '2020-05-25'), 2),
 ((1, '2020-05-23'), 2),
 ((2, '2020-05-25'), 3),
 ((4, '2020-05-26'), 3),
 ((11, '2020-05-23'), 1),
 ((2, '2020-05-24'), 2),
 ((10, '2020-05-23'), 2),
 ((3, '2020-05-26'), 2),
 ((1, '2020-05-24'), 3)]

In [47]:
# armo otro rdd de canciones donde fecha sea el dia anterior

def getDayBefore(x):
    list1 = ['02','04','06','09','11']
    list2 = ['01','05','07','08','10','12']
    ogDate = x[0].split('-')
    if ((ogDate[2] == '01') & (ogDate[1] in list1)):
        day = '31'
        month = str(int(ogDate[1])-1)
    elif ((ogDate[2] == '01') & (ogDate[1] in list2)):
        day = '30'
        month = str(int(ogDate[1])-1)
    elif ((ogDate[2] == '01') & (ogDate[1] == '03')):
        day = '28'
        month = str(int(ogDate[1])-1)
    else:
        day = str(int(ogDate[2]) - 1)
        month = ogDate[1]

    if ((ogDate[1] == '01') & (ogDate[2] == '01')):
        month = '12'
        year = str(int(ogDate[0])-1)
    else:
        year = ogDate[0]
        
    dayBefore = year+'-'+month+'-'+day
    return ((x[2], dayBefore),1)

cancionesRddDiaAnterior = cancionesRdd.map(getDayBefore)
cancionesRddDiaAnterior.collect()

[((1, '2020-05-22'), 1),
 ((10, '2020-05-22'), 1),
 ((1, '2020-05-22'), 1),
 ((11, '2020-05-22'), 1),
 ((10, '2020-05-22'), 1),
 ((2, '2020-05-23'), 1),
 ((1, '2020-05-23'), 1),
 ((1, '2020-05-23'), 1),
 ((1, '2020-05-23'), 1),
 ((2, '2020-05-23'), 1),
 ((3, '2020-05-24'), 1),
 ((2, '2020-05-24'), 1),
 ((2, '2020-05-24'), 1),
 ((2, '2020-05-24'), 1),
 ((3, '2020-05-24'), 1),
 ((4, '2020-05-25'), 1),
 ((3, '2020-05-25'), 1),
 ((4, '2020-05-25'), 1),
 ((3, '2020-05-25'), 1),
 ((4, '2020-05-25'), 1)]

In [48]:
cancionesRddAnt = cancionesRddDiaAnterior.reduceByKey(lambda x,y:x+y)
cancionesRddAnt.collect()

[((3, '2020-05-25'), 2),
 ((1, '2020-05-22'), 2),
 ((1, '2020-05-23'), 3),
 ((11, '2020-05-22'), 1),
 ((4, '2020-05-25'), 3),
 ((2, '2020-05-24'), 3),
 ((3, '2020-05-24'), 2),
 ((10, '2020-05-22'), 2),
 ((2, '2020-05-23'), 2)]

In [49]:
joined = cancionesRddReduced.join(cancionesRddAnt)
joined.collect()

[((3, '2020-05-25'), (2, 2)),
 ((1, '2020-05-23'), (2, 3)),
 ((2, '2020-05-24'), (2, 3))]

In [52]:
risingSongs = joined.filter(lambda x: x[1][1]>x[1][0]).map(lambda x: (x[0][0],'rising'))
risingSongs.collect()

[(1, 'rising'), (2, 'rising')]

In [71]:
updatedTags = tagsRdd.leftOuterJoin(risingSongs)\
.flatMap(lambda x: [(x[0], tag) for tag in x[1]])\
.filter(lambda x: x[1] is not None)
updatedTags.collect()

[(1, 'pop'),
 (1, 'rising'),
 (1, 'britney spears'),
 (1, 'rising'),
 (2, 'acoustic'),
 (2, 'rising'),
 (2, 'britney spears'),
 (2, 'rising'),
 (3, 'pop'),
 (3, '2010s'),
 (4, 'pop'),
 (4, 'britney spears'),
 (4, '2010s'),
 (10, 'actual'),
 (10, 'pop'),
 (10, 'top10'),
 (10, 'rising'),
 (11, '2010s')]